In [1]:
##addReconkey

import pandas as pd
import glob
import ast
import numpy as np
import os
import re

def addReconKey(inFile, outFile, keyFieldList):
    """
    addReconKey
    inserts unique key at first position based on reconKey file specification  
    """
    #print("inFile : %s" % inFile)
    # csv_input = pd.read_csv(open(inFile, encoding="utf8", errors='ignore'), skiprows=[1], sep="|")
    csv_input = pd.read_csv(open(inFile, encoding="utf8", errors='ignore'), sep="|")
    # drop row with condition
    # csv_input.drop(csv_input[csv_input[keyFieldList[0]] < 5].index, inplace = True)
    #reconKey = csv_input[['U2', 'U3']].astype(str).agg('#'.join, axis=1)
    try:
        reconKeyDirty = csv_input[keyFieldList].replace(np.nan, '').astype(str).agg('#'.join, axis=1)
        reconKey = [ re.sub(r".0#", "#", e) for e in reconKeyDirty ]
    except:
        print('sory, no match with key field list: %s for file %s' % (keyFieldList, inFile) )
        return 1
    idx = 0
    csv_input.insert(loc=idx, column='RECON_KEY', value=reconKey)
    if os.path.exists(outFile):
        os.remove(outFile)
    csv_input.to_csv(outFile, sep=";", index=False)

def readConfigFiles(reconKeyFile):
    # print("Reading congig files : %s\n" % (reconKeyFile))
    reconKeyDic = {}
    fileReconKeyDic = open(reconKeyFile, "r", encoding="utf8", errors='ignore')
    contents = fileReconKeyDic.read()
    reconKeyDic = ast.literal_eval(contents)
    fileReconKeyDic.close()
    # print(type(reconKeyDic))
    # print(reconKeyDic)
    return reconKeyDic

reconKeyDic = {}
reconKeyList = []
reconKeyFilePath = '/Users/e5527h6/Documents/Programming/python/HlbRecon_MX3_1Plus/config/reconKey.dic'
# MX3
inPath = '/Users/e5527h6/Documents/Programming/data/MCC'
csvPath = os.path.join(inPath, '*.csv')
outPath = os.path.join(inPath, 'Out')
#for filename in os.listdir(inPath):
for filename in glob.glob(csvPath):
    #print("filename : %s" % filename)
    #inFilePath = os.path.join(inPath, filename)
    inFilePath = filename
   # keyFieldList = ['U2', 'U3', 'U4']
    cleanNameInFile = re.sub(r"_\d+.csv$", ".csv", os.path.basename(filename))
    outFilePath = os.path.join(outPath, cleanNameInFile)
    reconKeyDic = readConfigFiles(reconKeyFilePath)
    if cleanNameInFile in reconKeyDic:
        keyFieldList = reconKeyDic[cleanNameInFile]
    else:
        print("File : %s not in reconKeyDic" % cleanNameInFile)
        continue
    addReconKey(inFilePath, outFilePath, keyFieldList)
   
print('Done')


Done
